In [1]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar']




In [2]:

"""Shows basic usage of the Google Calendar API.
Prints the start and name of the next 10 events on the user's calendar.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
calendar_token_loc = 'creds_google/calendar-token.pickle'
credentials_loc = 'creds_google/credentials.json'

if os.path.exists(calendar_token_loc):
    with open(calendar_token_loc, 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            credentials_loc, SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open(calendar_token_loc, 'wb') as token:
        pickle.dump(creds, token)

service = build('calendar', 'v3', credentials=creds)
# Call the Calendar API
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId='primary', timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()

events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])



Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=601636917597-efhlajhkhltiie45u6p0lqscap6fomtd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A41113%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=XtNRR8lJ6hXobLpA71jNjNhtxt3BQl&access_type=offline
Getting the upcoming 10 events
2021-01-18T19:00:00+03:00 Wolf Pack Meeting <3
2021-01-18T22:00:00+03:00 IBM Data Science
2021-01-19T15:00:00+03:00 LCM DRYRUNS
2021-01-19T18:00:00+03:00 Closing O2O | Zainab <3
2021-01-19T22:00:00+03:00 IBM Data Science
2021-01-20T15:30:00+03:00 MLSD 
2021-01-20T22:00:00+03:00 IBM Data Science
2021-01-21T11:00:00+03:00 Work : Product Call
2021-01-21T15:00:00+03:00 Closing O2O | Sara <3
2021-01-21T16:00:00+03:00 LC CLOSING REPORT MEETING


In [4]:
_ = {
    "attachments": [ # File attachments for the event. Currently only Google Drive attachments are supported.
        # In order to modify attachments the supportsAttachments request parameter should be set to True.
        # There can be at most 25 attachments per event,
      {
        "mimeType": "A String", # Internet media type (MIME type) of the attachment.
        "title": "A String", # Attachment title.
        "fileUrl": "A String", # URL link to the attachment.
            # For adding Google Drive file attachments use the same format as in alternateLink property of the Files resource in the Drive API.
            # Required when adding an attachment.
        "iconLink": "A String", # URL link to the attachment's icon. Read-only.
        "fileId": "A String", # ID of the attached file. Read-only.
            # For Google Drive files, this is the ID of the corresponding Files resource entry in the Drive API.
      },
    ],
    "creator": { # The creator of the event. Read-only.
      "self": False, # Whether the creator corresponds to the calendar on which this copy of the event appears. Read-only. The default is False.
      "displayName": "A String", # The creator's name, if available.
      "email": "A String", # The creator's email address, if available.
      "id": "A String", # The creator's Profile ID, if available. It corresponds to the id field in the People collection of the Google+ API
    },
    "organizer": { # The organizer of the event. If the organizer is also an attendee, this is indicated with a separate entry in attendees with the organizer field set to True. To change the organizer, use the move operation. Read-only, except when importing an event.
      "self": False, # Whether the organizer corresponds to the calendar on which this copy of the event appears. Read-only. The default is False.
      "displayName": "A String", # The organizer's name, if available.
      "email": "A String", # The organizer's email address, if available. It must be a valid email address as per RFC5322.
      "id": "A String", # The organizer's Profile ID, if available. It corresponds to the id field in the People collection of the Google+ API
    },
    "summary": "A String", # Title of the event.
    "id": "A String", # Opaque identifier of the event. When creating new single or recurring events, you can specify their IDs. Provided IDs must follow these rules:
        # - characters allowed in the ID are those used in base32hex encoding, i.e. lowercase letters a-v and digits 0-9, see section 3.1.2 in RFC2938
        # - the length of the ID must be between 5 and 1024 characters
        # - the ID must be unique per calendar  Due to the globally distributed nature of the system, we cannot guarantee that ID collisions will be detected at event creation time. To minimize the risk of collisions we recommend using an established UUID algorithm such as one described in RFC4122.
        # If you do not specify an ID, it will be automatically generated by the server.
        # Note that the icalUID and the id are not identical and only one of them should be supplied at event creation time. One difference in their semantics is that in recurring events, all occurrences of one event have different ids while they all share the same icalUIDs.
    "hangoutLink": "A String", # An absolute link to the Google+ hangout associated with this event. Read-only.
    "attendees": [ # The attendees of the event. See the Events with attendees guide for more information on scheduling events with other calendar users.
      {
        "comment": "A String", # The attendee's response comment. Optional.
        "displayName": "A String", # The attendee's name, if available. Optional.
        "responseStatus": "A String", # The attendee's response status. Possible values are:
            # - "needsAction" - The attendee has not responded to the invitation.
            # - "declined" - The attendee has declined the invitation.
            # - "tentative" - The attendee has tentatively accepted the invitation.
            # - "accepted" - The attendee has accepted the invitation.
        "self": False, # Whether this entry represents the calendar on which this copy of the event appears. Read-only. The default is False.
        "id": "A String", # The attendee's Profile ID, if available. It corresponds to the id field in the People collection of the Google+ API
        "additionalGuests": 0, # Number of additional guests. Optional. The default is 0.
        "resource": False, # Whether the attendee is a resource. Can only be set when the attendee is added to the event for the first time. Subsequent modifications are ignored. Optional. The default is False.
        "organizer": True or False, # Whether the attendee is the organizer of the event. Read-only. The default is False.
        "optional": False, # Whether this is an optional attendee. Optional. The default is False.
        "email": "A String", # The attendee's email address, if available. This field must be present when adding an attendee. It must be a valid email address as per RFC5322.
            # Required when adding an attendee.
      },
    ],
    "start": { # The (inclusive) start time of the event. For a recurring event, this is the start time of the first instance.
      "date": "A String", # The date, in the format "yyyy-mm-dd", if this is an all-day event.
      "timeZone": "A String", # The time zone in which the time is specified. (Formatted as an IANA Time Zone Database name, e.g. "Europe/Zurich".) For recurring events this field is required and specifies the time zone in which the recurrence is expanded. For single events this field is optional and indicates a custom time zone for the event start/end.
      "dateTime": "A String", # The time, as a combined date-time value (formatted according to RFC3339). A time zone offset is required unless a time zone is explicitly specified in timeZone.
    },
    "htmlLink": "A String", # An absolute link to this event in the Google Calendar Web UI. Read-only.
    "recurrence": [ # List of RRULE, EXRULE, RDATE and EXDATE lines for a recurring event, as specified in RFC5545. Note that DTSTART and DTEND lines are not allowed in this field; event start and end times are specified in the start and end fields. This field is omitted for single events or instances of recurring events.
      "A String",
    ],
    "source": { # Source from which the event was created. For example, a web page, an email message or any document identifiable by an URL with HTTP or HTTPS scheme. Can only be seen or modified by the creator of the event.
      "url": "A String", # URL of the source pointing to a resource. The URL scheme must be HTTP or HTTPS.
      "title": "A String", # Title of the source; for example a title of a web page or an email subject.
    },
    "etag": "A String", # ETag of the resource.
    "location": "A String", # Geographic location of the event as free-form text. Optional.
    "recurringEventId": "A String", # For an instance of a recurring event, this is the id of the recurring event to which this instance belongs. Immutable.
    "gadget": { # A gadget that extends this event.
      "preferences": { # Preferences.
        "a_key": "A String", # The preference name and corresponding value.
      },
      "title": "A String", # The gadget's title.
      "height": 42, # The gadget's height in pixels. The height must be an integer greater than 0. Optional.
      "width": 42, # The gadget's width in pixels. The width must be an integer greater than 0. Optional.
      "link": "A String", # The gadget's URL. The URL scheme must be HTTPS.
      "type": "A String", # The gadget's type.
      "display": "A String", # The gadget's display mode. Optional. Possible values are:
          # - "icon" - The gadget displays next to the event's title in the calendar view.
          # - "chip" - The gadget displays when the event is clicked.
      "iconLink": "A String", # The gadget's icon URL. The URL scheme must be HTTPS.
    },
    "status": "A String", # Status of the event. Optional. Possible values are:
        # - "confirmed" - The event is confirmed. This is the default status.
        # - "tentative" - The event is tentatively confirmed.
        # - "cancelled" - The event is cancelled (deleted). The list method returns cancelled events only on incremental sync (when syncToken or updatedMin are specified) or if the showDeleted flag is set to True. The get method always returns them.
        # A cancelled status represents two different states depending on the event type:
        # - Cancelled exceptions of an uncancelled recurring event indicate that this instance should no longer be presented to the user. Clients should store these events for the lifetime of the parent recurring event.
        # Cancelled exceptions are only guaranteed to have values for the id, recurringEventId and originalStartTime fields populated. The other fields might be empty.
        # - All other cancelled events represent deleted events. Clients should remove their locally synced copies. Such cancelled events will eventually disappear, so do not rely on them being available indefinitely.
        # Deleted events are only guaranteed to have the id field populated.   On the organizer's calendar, cancelled events continue to expose event details (summary, location, etc.) so that they can be restored (undeleted). Similarly, the events to which the user was invited and that they manually removed continue to provide details. However, incremental sync requests with showDeleted set to False will not return these details.
        # If an event changes its organizer (for example via the move operation) and the original organizer is not on the attendee list, it will leave behind a cancelled event where only the id field is guaranteed to be populated.
    "updated": "A String", # Last modification time of the event (as a RFC3339 timestamp). Read-only.
    "description": "A String", # Description of the event. Can contain HTML. Optional.
    "iCalUID": "A String", # Event unique identifier as defined in RFC5545. It is used to uniquely identify events accross calendaring systems and must be supplied when importing events via the import method.
        # Note that the icalUID and the id are not identical and only one of them should be supplied at event creation time. One difference in their semantics is that in recurring events, all occurrences of one event have different ids while they all share the same icalUIDs.
    "extendedProperties": { # Extended properties of the event.
      "shared": { # Properties that are shared between copies of the event on other attendees' calendars.
        "a_key": "A String", # The name of the shared property and the corresponding value.
      },
      "private": { # Properties that are private to the copy of the event that appears on this calendar.
        "a_key": "A String", # The name of the private property and the corresponding value.
      },
    },
    "endTimeUnspecified": False, # Whether the end time is actually unspecified. An end time is still provided for compatibility reasons, even if this attribute is set to True. The default is False.
    "sequence": 42, # Sequence number as per iCalendar.
    "visibility": "default", # Visibility of the event. Optional. Possible values are:
        # - "default" - Uses the default visibility for events on the calendar. This is the default value.
        # - "public" - The event is public and event details are visible to all readers of the calendar.
        # - "private" - The event is private and only event attendees may view event details.
        # - "confidential" - The event is private. This value is provided for compatibility reasons.
    "guestsCanModify": False, # Whether attendees other than the organizer can modify the event. Optional. The default is False.
    "end": { # The (exclusive) end time of the event. For a recurring event, this is the end time of the first instance.
      "date": "A String", # The date, in the format "yyyy-mm-dd", if this is an all-day event.
      "timeZone": "A String", # The time zone in which the time is specified. (Formatted as an IANA Time Zone Database name, e.g. "Europe/Zurich".) For recurring events this field is required and specifies the time zone in which the recurrence is expanded. For single events this field is optional and indicates a custom time zone for the event start/end.
      "dateTime": "A String", # The time, as a combined date-time value (formatted according to RFC3339). A time zone offset is required unless a time zone is explicitly specified in timeZone.
    },
    "attendeesOmitted": False, # Whether attendees may have been omitted from the event's representation. When retrieving an event, this may be due to a restriction specified by the maxAttendee query parameter. When updating an event, this can be used to only update the participant's response. Optional. The default is False.
    "kind": "calendar#event", # Type of the resource ("calendar#event").
    "locked": False, # Whether this is a locked event copy where no changes can be made to the main event fields "summary", "description", "location", "start", "end" or "recurrence". The default is False. Read-Only.
    "anyoneCanAddSelf": False, # Whether anyone can invite themselves to the event (currently works for Google+ events only). Optional. The default is False.
    "colorId": "A String", # The color of the event. This is an ID referring to an entry in the event section of the colors definition (see the  colors endpoint). Optional.
    "created": "A String", # Creation time of the event (as a RFC3339 timestamp). Read-only.
    "reminders": { # Information about the event's reminders for the authenticated user.
      "overrides": [ # If the event doesn't use the default reminders, this lists the reminders specific to the event, or, if not set, indicates that no reminders are set for this event. The maximum number of override reminders is 5.
        {
          "minutes": 42, # Number of minutes before the start of the event when the reminder should trigger. Valid values are between 0 and 40320 (4 weeks in minutes).
              # Required when adding a reminder.
          "method": "A String", # The method used by this reminder. Possible values are:
              # - "email" - Reminders are sent via email.
              # - "popup" - Reminders are sent via a UI popup.
              # Required when adding a reminder.
        },
      ],
      "useDefault": True or False, # Whether the default reminders of the calendar apply to the event.
    },
    "guestsCanSeeOtherGuests": True, # Whether attendees other than the organizer can see who the event's attendees are. Optional. The default is True.
    "originalStartTime": { # For an instance of a recurring event, this is the time at which this event would start according to the recurrence data in the recurring event identified by recurringEventId. It uniquely identifies the instance within the recurring event series even if the instance was moved to a different time. Immutable.
      "date": "A String", # The date, in the format "yyyy-mm-dd", if this is an all-day event.
      "timeZone": "A String", # The time zone in which the time is specified. (Formatted as an IANA Time Zone Database name, e.g. "Europe/Zurich".) For recurring events this field is required and specifies the time zone in which the recurrence is expanded. For single events this field is optional and indicates a custom time zone for the event start/end.
      "dateTime": "A String", # The time, as a combined date-time value (formatted according to RFC3339). A time zone offset is required unless a time zone is explicitly specified in timeZone.
    },
    "guestsCanInviteOthers": True, # Whether attendees other than the organizer can invite others to the event. Optional. The default is True.
    "transparency": "opaque", # Whether the event blocks time on the calendar. Optional. Possible values are:
        # - "opaque" - Default value. The event does block time on the calendar. This is equivalent to setting Show me as to Busy in the Calendar UI.
        # - "transparent" - The event does not block time on the calendar. This is equivalent to setting Show me as to Available in the Calendar UI.
    "privateCopy": False, # If set to True, Event propagation is disabled. Note that it is not the same thing as Private event properties. Optional. Immutable. The default is False.
    "conferenceData": { # The conference-related information, such as details of a Hangouts Meet conference. To create new conference details use the createRequest field. To persist your changes, remember to set the conferenceDataVersion request parameter to 1 for all event modification requests.
      "entryPoints": [ # Information about individual conference entry points, such as URLs or phone numbers.
          # All of them must belong to the same conference.
          # Either conferenceSolution and at least one entryPoint, or createRequest is required.
        {
          "password": "A String", # The password to access the conference. The maximum length is 128 characters.
              # When creating new conference data, populate only the subset of {meetingCode, accessCode, passcode, password, pin} fields that match the terminology that the conference provider uses. Only the populated fields should be displayed.
              # Optional.
          "meetingCode": "A String", # The meeting code to access the conference. The maximum length is 128 characters.
              # When creating new conference data, populate only the subset of {meetingCode, accessCode, passcode, password, pin} fields that match the terminology that the conference provider uses. Only the populated fields should be displayed.
              # Optional.
          "pin": "A String", # The PIN to access the conference. The maximum length is 128 characters.
              # When creating new conference data, populate only the subset of {meetingCode, accessCode, passcode, password, pin} fields that match the terminology that the conference provider uses. Only the populated fields should be displayed.
              # Optional.
          "uri": "A String", # The URI of the entry point. The maximum length is 1300 characters.
              # Format:
              # - for video, http: or https: schema is required.
              # - for phone, tel: schema is required. The URI should include the entire dial sequence (e.g., tel:+12345678900,,,123456789;1234).
              # - for sip, sip: schema is required, e.g., sip:12345678@myprovider.com.
              # - for more, http: or https: schema is required.
          "entryPointFeatures": [ # Features of the entry point, such as being toll or toll-free. One entry point can have multiple features. However, toll and toll-free cannot be both set on the same entry point.
            "A String",
          ],
          "label": "A String", # The label for the URI. Visible to end users. Not localized. The maximum length is 512 characters.
              # Examples:
              # - for video: meet.google.com/aaa-bbbb-ccc
              # - for phone: +1 123 268 2601
              # - for sip: 12345678@altostrat.com
              # - for more: should not be filled
              # Optional.
          "entryPointType": "A String", # The type of the conference entry point.
              # Possible values are:
              # - "video" - joining a conference over HTTP. A conference can have zero or one video entry point.
              # - "phone" - joining a conference by dialing a phone number. A conference can have zero or more phone entry points.
              # - "sip" - joining a conference over SIP. A conference can have zero or one sip entry point.
              # - "more" - further conference joining instructions, for example additional phone numbers. A conference can have zero or one more entry point. A conference with only a more entry point is not a valid conference.
          "regionCode": "A String", # The CLDR/ISO 3166 region code for the country associated with this phone access. Example: "SE" for Sweden.
              # Calendar backend will populate this field only for EntryPointType.PHONE.
          "passcode": "A String", # The passcode to access the conference. The maximum length is 128 characters.
              # When creating new conference data, populate only the subset of {meetingCode, accessCode, passcode, password, pin} fields that match the terminology that the conference provider uses. Only the populated fields should be displayed.
          "accessCode": "A String", # The access code to access the conference. The maximum length is 128 characters.
              # When creating new conference data, populate only the subset of {meetingCode, accessCode, passcode, password, pin} fields that match the terminology that the conference provider uses. Only the populated fields should be displayed.
              # Optional.
        },
      ],
      "parameters": { # Additional properties related to a conference. An example would be a solution-specific setting for enabling video streaming.
        "addOnParameters": { # Additional add-on specific data.
          "parameters": {
            "a_key": "A String",
          },
        },
      },
      "notes": "A String", # Additional notes (such as instructions from the domain administrator, legal notices) to display to the user. Can contain HTML. The maximum length is 2048 characters. Optional.
      "conferenceId": "A String", # The ID of the conference.
          # Can be used by developers to keep track of conferences, should not be displayed to users.
          # Values for solution types:
          # - "eventHangout": unset.
          # - "eventNamedHangout": the name of the Hangout.
          # - "hangoutsMeet": the 10-letter meeting code, for example "aaa-bbbb-ccc".
          # - "addOn": defined by 3P conference provider.  Optional.
      "createRequest": { # A request to generate a new conference and attach it to the event. The data is generated asynchronously. To see whether the data is present check the status field.
          # Either conferenceSolution and at least one entryPoint, or createRequest is required.
        "status": { # The status of the conference create request.
          "statusCode": "A String", # The current status of the conference create request. Read-only.
              # The possible values are:
              # - "pending": the conference create request is still being processed.
              # - "success": the conference create request succeeded, the entry points are populated.
              # - "failure": the conference create request failed, there are no entry points.
        },
        "conferenceSolutionKey": { # The conference solution, such as Hangouts or Hangouts Meet.
          "type": "A String", # The conference solution type.
              # If a client encounters an unfamiliar or empty type, it should still be able to display the entry points. However, it should disallow modifications.
              # The possible values are:
              # - "eventHangout" for Hangouts for consumers (http://hangouts.google.com)
              # - "eventNamedHangout" for classic Hangouts for G Suite users (http://hangouts.google.com)
              # - "hangoutsMeet" for Hangouts Meet (http://meet.google.com)
              # - "addOn" for 3P conference providers
        },
        "requestId": "A String", # The client-generated unique ID for this request.
            # Clients should regenerate this ID for every new request. If an ID provided is the same as for the previous request, the request is ignored.
      },
      "conferenceSolution": { # The conference solution, such as Hangouts or Hangouts Meet.
          # Unset for a conference with a failed create request.
          # Either conferenceSolution and at least one entryPoint, or createRequest is required.
        "iconUri": "A String", # The user-visible icon for this solution.
        "name": "A String", # The user-visible name of this solution. Not localized.
        "key": { # The key which can uniquely identify the conference solution for this event.
          "type": "A String", # The conference solution type.
              # If a client encounters an unfamiliar or empty type, it should still be able to display the entry points. However, it should disallow modifications.
              # The possible values are:
              # - "eventHangout" for Hangouts for consumers (http://hangouts.google.com)
              # - "eventNamedHangout" for classic Hangouts for G Suite users (http://hangouts.google.com)
              # - "hangoutsMeet" for Hangouts Meet (http://meet.google.com)
              # - "addOn" for 3P conference providers
        },
      },
      "signature": "A String", # The signature of the conference data.
          # Generated on server side. Must be preserved while copying the conference data between events, otherwise the conference data will not be copied.
          # Unset for a conference with a failed create request.
          # Optional for a conference with a pending create request.
    },
  }

In [28]:
print(_.keys())
_['start']

dict_keys(['attachments', 'creator', 'organizer', 'summary', 'id', 'hangoutLink', 'attendees', 'start', 'htmlLink', 'recurrence', 'source', 'etag', 'location', 'recurringEventId', 'gadget', 'status', 'updated', 'description', 'iCalUID', 'extendedProperties', 'endTimeUnspecified', 'sequence', 'visibility', 'guestsCanModify', 'end', 'attendeesOmitted', 'kind', 'locked', 'anyoneCanAddSelf', 'colorId', 'created', 'reminders', 'guestsCanSeeOtherGuests', 'originalStartTime', 'guestsCanInviteOthers', 'transparency', 'privateCopy', 'conferenceData'])


{'date': 'A String', 'timeZone': 'A String', 'dateTime': 'A String'}

In [6]:
event = {
  'summary': 'Appointment',
  'location': 'Somewhere',
  'start': {
    'dateTime': '2021-01-20T10:00:00.000',
    'timeZone': 'GMT+3:00'
  },
  'end': {
    'dateTime': '2021-01-20T10:25:00.000',
      'timeZone': 'GMT+3:00'
  },
  'recurrence': [
    'RRULE:FREQ=DAILY;UNTIL=20210122T235959Z',
  ],
}
recurring_event = service.events().insert(calendarId='primary', body=event).execute()

print( recurring_event['id'])

hh3tv9od5rbpd0jue2dib69ssc
